In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Basic packages...!!!

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,CuDNNLSTM,GlobalMaxPool1D,Bidirectional

# Importing Dataset

In [ ]:
train = pd.read_csv("../input/quora-insincere-questions-classification/train.csv")
test = pd.read_csv("../input/quora-insincere-questions-classification/test.csv")
submission = pd.read_csv("../input/quora-insincere-questions-classification/sample_submission.csv")

importing test id's for futher use

In [ ]:
id = test['qid']

In [ ]:
print('There are {} rows and {} columns in train'.format(train.shape[0],train.shape[1]))
print('There are {} rows and {} columns in test'.format(test.shape[0],test.shape[1]))

In [ ]:
x=train.target.value_counts()
sns.barplot(x.index,x)
plt.gca().set_ylabel('samples')

 # Text Preprocessing...!

In [ ]:
train.head()

In [ ]:
train.rename(columns={'question_text' : 'text'},inplace = True)

In [ ]:
train['target'].value_counts()

Here the target is classified in two section

* 1 : Insincere(An insincere question is defined as a question intended to make a statement rather than look for helpful answers)

* 0 : sincere(Its the question which is helpful

now we will look at moissing values...

In [ ]:
train.isnull().sum()

Ahhyeah....their is none of missing values present

here column named 'qid' doesn't play any significant role in training the model....so,we will drop it

In [ ]:
train.drop('qid',inplace = True,axis = 1)

In [ ]:
train.head()

now, we will apply the same step to the test dataset

In [ ]:
test.head()


In [ ]:
test.rename(columns={'question_text' : 'text'},inplace = True)

In [ ]:
test.isnull().sum()

In [ ]:
test.drop('qid',inplace = True,axis = 1)

In [ ]:
test.head()

now let's begin to preprocess out data through which one can train it or test it efficiently

for data preprocessing...:
reference : ##https://www.kaggle.com/saranyashalya/getting-started-with-text-preprocessing-nlp##

importing some basic Inbuild functions 

In [ ]:
train.head()

In [ ]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_urls(text):
    return re.sub(r'http\S+', '', text)
# Removing hashtags
def remove_hash(text):
    text = " ".join(word.strip() for word in re.split('#|_', text))
    return text
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_urls(text)
    text = remove_hash(text)
    text = remove_stopwords(text)
    return text

now applying this user - defined fuction to our dataset (i.e train and test set)

In [ ]:
train['text']=train['text'].apply(denoise_text)


In [ ]:
test['text']=test['text'].apply(denoise_text)

now we will break the sentences into meaningful tokens...!!

but first we will set basic parameters

In [ ]:
max_features = 20000
maxlen = 100

In [ ]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(train.text)
tokenized_train = tokenizer.texts_to_sequences(train.text)
X = sequence.pad_sequences(tokenized_train, maxlen=maxlen)

In [ ]:
tokenized_test = tokenizer.texts_to_sequences(test.text)
sub_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)

In [ ]:
X.shape

In [ ]:
sub_test.shape

# Text Modelling

now we will split the dataset by train_test_split

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test ,y_train ,y_test = train_test_split(X,train['target'])

fiting the dataset to a logit model...!! 

In [ ]:
from sklearn.linear_model import LogisticRegression
logit = LogisticRegression(max_iter = 1000)
logit.fit(X_train,y_train)


now its time to check the model score...!!

In [ ]:
score = logit.score(X_train,y_train)

In [ ]:
score

AAAHyeah.....93% train accuracy...have we overfitted our model....to check for it we will test our model to unknown dataset

In [ ]:
y = logit.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print("accuracy of X_test data is : ",accuracy_score(y_test,y))

nice..!! we have a accuracy of more than 93% 

In [ ]:
#classification report
from sklearn.metrics import classification_report
print("classification_report of X_test data is : ",classification_report(y_test,y))

**If you like my kernal please upvote...!!!!**

In [ ]:
final = logit.predict(sub_test)

In [ ]:
final.shape

In [ ]:
submission = pd.DataFrame({
        "qid": id,
        "prediction": final
    })

In [ ]:
submission.to_csv("samplesubmission.csv", index=False)

In [ ]:
sub = pd.read_csv('samplesubmission.csv')

In [ ]:
sub.head()